In [1]:
import ray
import time
import numpy as np

In [2]:
ray.init()

2022-08-15 11:19:19,733	INFO services.py:1456 -- View the Ray dashboard at http://127.0.0.1:8265


RayContext(dashboard_url='127.0.0.1:8265', python_version='3.9.12', ray_version='1.12.0', ray_commit='f18fc31c7562990955556899090f8e8656b48d2d', address_info={'node_ip_address': '172.31.251.194', 'raylet_ip_address': '172.31.251.194', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-08-15_11-19-17_875971_13918/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-08-15_11-19-17_875971_13918/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-08-15_11-19-17_875971_13918', 'metrics_export_port': 48623, 'gcs_address': '172.31.251.194:53273', 'address': '172.31.251.194:53273', 'node_id': '10fe10bb191eb1f6f0572e6bcfd073daac55e20fb62a7ee5735417b8'})

In [3]:
@ray.remote
def f(x):
    time.sleep(1)
    return x

# Start 4 tasks in parallel.
result_ids = []
for i in range(4):
    result_ids.append(f.remote(i))
    
# Wait for the tasks to complete and retrieve the results.
# With at least 4 cores, this will take 1 second.
results = ray.get(result_ids)  # [0, 1, 2, 3]

In [4]:
results

[0, 1, 2, 3]

In [5]:
@ray.remote
def create_matrix(size):
    return np.random.normal(size=size)

@ray.remote
def multiply_matrices(x, y):
    return np.dot(x, y)

x_id = create_matrix.remote([1000, 1000])
y_id = create_matrix.remote([1000, 1000])
z_id = multiply_matrices.remote(x_id, y_id)

# Get the results.
z = ray.get(z_id)

In [6]:
z

array([[ 11.37093904, -44.50457117,  -6.15293878, ..., -27.40695253,
        -34.34495728, -30.85410735],
       [ 24.14780352,  -8.60058469,  -6.05974163, ...,  27.91909271,
        -21.98551581,  17.51412485],
       [-53.53439274,  -5.21069163, -27.29399781, ..., -50.50875694,
        -15.00434003,  -7.74149601],
       ...,
       [ 11.63091135, -42.5206375 ,  41.59662256, ...,  -7.74239924,
        -21.49065858,   2.40885547],
       [-41.03914363, -12.58284965,   8.20308365, ...,  17.73708442,
          6.68521548,  13.44754728],
       [-15.13829411,  -4.22607731,  27.92421435, ...,  41.96081871,
         36.10455792,   8.85761925]])

In [8]:
# from ray import tune


# def objective(x, a, b):  # Define an objective function.
#     return a * (x ** 0.5) + b


# def trainable(config):  # Pass a "config" dictionary into your trainable.

#     for x in range(20):  # "Train" for 20 iterations and compute intermediate scores.
#         score = objective(x, config["a"], config["b"])

#         tune.report(score=score)  # Send the score to Tune.
